## Задача 3-2. Задача TSP: нижняя оценка Гельда—Карпа.

В этой задаче Вам предлагается релизовать алгоритм Гельда—Карпа для нижней оценки стоимости решения в задаче Euclidean TSP.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP. Это в точности те же входные данные, что и в задании 3-1.
* Реализуйте функцию `lower_bound_tsp`. При этом можно пользоваться каким-нибудь стандартным алгоритмом построения минимального остовного дерева из библиотеки [`networkx`](https://networkx.github.io/), входящей в состав дистрибутива Anaconda.
* Запустите функцию `run_all()`, чтобы протестировать свой код, и напишите полученные, как следствия, верхние оценки погрешностей решений, которые были получены Вашими алгоритмами NN и NI при решении задания 3-1. Запишите свои выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [23]:
from typing import List, Tuple
from math import sqrt
from itertools import combinations, islice
import numpy as np


def read_tsp_instance(filename: str) -> List[Tuple[int,int]]:
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0, 0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates


def euclidean_distance(point1: Tuple[int,int], point2: Tuple[int,int]) -> float:
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)

In [24]:
import networkx as nx

In [35]:
def lower_bound_tsp(vertex_coordinates: List[Tuple[int,int]]) -> float:
    
    ans = 0
    
    x = vertex_coordinates
    v_c = range(len(x))
    I = np.zeros(len(vertex_coordinates), dtype='int64')
    
    time_start = time.monotonic()
    
    while time.monotonic() - time_start < 60:
        
        nw = sum(I) * 2
        g = nx.Graph()
        
        # nodes
        g.add_nodes_from(v_c)
        
        # edges
        for a, b in combinations(v_c, 2):
            d = I[a] + I[b]
            w = euclidean_distance(x[a], x[b]) - d
            g.add_edge(a, b, weight=w)
        
        
        min_ostov = nx.minimum_spanning_tree(g)
        
        bound = nw + min_ostov.size('weight')
        if ans < bound:
            ans = bound
        
        tmp = np.zeros(min_ostov.number_of_nodes(), dtype='int64')
        
        for i in range(len(tmp)):
            tmp[i] = 1 - min_ostov.degree(i) + (I[i] + 1)
            
        I = tmp
        
        
    return ans

In [36]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['pr107.tsp', 'pr152.tsp', 'd198.tsp', 'pr439.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp']
    for x in instance_filenames:
        filename = 'tsp-instances/' + x
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        bound = lower_bound_tsp(instance)
        time_nn = time.monotonic()-time_start
        print(' done in {:.2} seconds with lower bound {}'.format(time_nn, int(bound)))

In [37]:
run_all()

Instance tsp-instances/pr107.tsp… done in 6e+01 seconds with lower bound 37492
Instance tsp-instances/pr152.tsp… done in 6e+01 seconds with lower bound 63182
Instance tsp-instances/d198.tsp… done in 6e+01 seconds with lower bound 13086
Instance tsp-instances/pr439.tsp… done in 6e+01 seconds with lower bound 95885
Instance tsp-instances/d493.tsp… done in 6e+01 seconds with lower bound 31567
Instance tsp-instances/d657.tsp… done in 6.1e+01 seconds with lower bound 45270
Instance tsp-instances/d2103.tsp… done in 8.1e+01 seconds with lower bound 76402


In [49]:
print('d198.tsp      18830  NN with error(1.5)       17587  NI with error(1.36)')
print('d493.tsp      44160  NN with error(1.41)      39982  NI with error(1.35)')
print('d657.tsp      62860  NN with error(1.4)       57906  NI with error(1.3)')
print('pr107.tsp     47464  NN with error(1.33)      52587  NI with error(1.24)')
print('pr152.tsp     85314  NN with error(1.43)      87848  NI with error(1.29)')
print('pr439.tsp     131702 NN with error(1.41)      130254 NI with error(1.4)')
print('pr103.tsp     91411  NN with error(1.2)       103887 NI with error(1.24)')

d198.tsp      18830  NN with error(1.5)       17587  NI with error(1.36)
d493.tsp      44160  NN with error(1.41)      39982  NI with error(1.35)
d657.tsp      62860  NN with error(1.4)       57906  NI with error(1.3)
pr107.tsp     47464  NN with error(1.33)      52587  NI with error(1.24)
pr152.tsp     85314  NN with error(1.43)      87848  NI with error(1.29)
pr439.tsp     131702 NN with error(1.41)      130254 NI with error(1.4)
pr103.tsp     91411  NN with error(1.2)       103887 NI with error(1.24)


## Выводы
Видим, что погрешность достаточно мала.